### Importing libraries

In [1]:
import openravepy
import trajoptpy
import json
import numpy as np
import trajoptpy.kin_utils as ku
from trajoptpy.check_traj import traj_is_safe
import time
import random 
import matplotlib.pyplot as plt
import pickle
from trajopt_util import *
from planning_util import *
from regression import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

FILENAME = 'data/base_planning/'

### Setting the trajopt

In [2]:
np.set_printoptions(precision=4, suppress=True)

env = openravepy.Environment()
env.StopSimulation()
env.Load("robots/pr2-beta-static.zae")
env.Load("../env/kitchen_counter.env.xml")
env.SetDefaultViewer()

True

### Setting the robot

In [3]:
robot = env.GetRobots()[0]
robot.SetTransform(np.array([[1.   ,0 ,  0.   ,  -0.25 ],
                             [ 0, 1.   ,  0.   ,  1.2],
                             [ 0.   ,  0.   ,  1.   ,  0.   ],
                             [ 0.   ,  0.   ,  0.   ,  1.   ]]))

leftarm_torso = robot.GetManipulator("leftarm_torso")
robot.SetDOFValues(np.array([0.261 , 0.5646,  1.1371, -0.65  , -2.1172,  2.0552, 0.99, -2.168 ]), \
                   leftarm_torso.GetArmIndices())

DOF = openravepy.DOFAffine   
robot.SetActiveDOFs([],DOF.X | DOF.Y | DOF.RotationAxis)
   
base_init = robot.GetActiveDOFValues()
base_target = base_init.copy()
base_target[1] -= 0.5
n_steps = 30
dof = len(base_init)
coeffs = [1., 1., 1.]

waypoint_left = np.array([0.7, 3,0])  
waypoint_right = np.array([0.7, -2.5,0])  

In [4]:
def generate_random_between(limit1, limit2):
    n = len(limit1)
    config = None
    while config is None:
        config = np.random.rand(n)
        config = np.multiply(config, limit2-limit1) + limit1
        robot.SetActiveDOFValues(config)
        if env.CheckCollision(robot) is True:
            continue
    return config

### Construct a random problem 

In [5]:
#setting the limit for base placements
init_limit1 = np.array([-1.25, -0.8, -np.pi])
init_limit2 = np.array([-0.25, 1.5, np.pi])

goal_limit1 = np.array([2.2, -0.8, -np.pi])
goal_limit2 = np.array([3.5, 1.5, np.pi])

### Build / Retrieve Dataset

In [6]:
def build_database(x_init, y_init, num_traj, limits, n_steps = 30, predictor=None):
    num_plan = 0
    comp_times = []
    costs = []

    tic = time.time()
    while len(x_init) < num_traj:
        print('{}th Planning, {} trajectories in database'.format(num_plan,len(x_init)))
        #Get a random pose as target
        init_joint = generate_random_between(limits['init'][0], limits['init'][1])
        target_joint = generate_random_between(limits['goal'][0], limits['goal'][1])
        robot.SetActiveDOFValues(init_joint)
        
        #Build memory using a set of predictors
        for func in predictor:
            #Predict the warmstart
            for waypoint in [waypoint_right,waypoint_left]:
                x_cur = np.concatenate([init_joint,target_joint])[None,:]

                if isinstance(func, Straight_Regressor):
                    traj,_ = func.predict_with_waypoint(init_joint, target_joint, waypoint, n_steps/2)     
                else:
                    traj,_ = func.predict(x_cur)
                traj = traj.reshape(-1,dof)
                traj[0,:] = init_joint

                #Plan
                robot.SetActiveDOFValues(init_joint)
                request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
                request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
                request_traj = set_init(request_traj, 'given_traj', traj)
                duration, result = run_opt(request_traj, env)    

                #Check traj result
                traj = result.GetTraj()
                if check_traj(env,result, target_joint):
                    print 'Planning is successfull!'
                    x_init.append(np.concatenate(x_cur))
                    y_init.append(traj.flatten())
                    comp_times.append(duration)
                    costs.append(result.GetCosts()[0][1])
                else:
                    print('Fail to find good solution!')     
                    
                num_plan += 1
        
    
    #Store the result
    toc = time.time()
    total_time = toc-tic
    success_rate = num_traj*1.0/num_plan
    x_init = np.vstack(x_init)
    y_init = np.vstack(y_init)
    
    data = dict()
    data['x'] = x_init
    data['y'] = y_init
    data['total_time'] = total_time
    data['success_rate'] = success_rate
    data['comp_times'] = comp_times
    data['costs'] = costs
    return data

In [32]:
to_build_database = False
to_augment_database = False
num_traj = 200
n_steps = 30
limits = dict()
limits['init'] = [init_limit1, init_limit2]
limits['goal'] = [goal_limit1, goal_limit2]

#build using straight-line warm-start
straight = Straight_Regressor(dof = dof, n_steps = n_steps)

if to_build_database:
    if to_augment_database:
        #load data
        data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
        x_init = list(data['x'])
        y_init = list(data['y'])
        num_traj += len(x_init)     
    else:
        x_init,y_init = [],[]
        
    data = build_database(x_init, y_init, num_traj,limits, n_steps = n_steps, predictor=[straight])
    x_init = data['x']
    y_init = data['y']
    pickle.dump(data,open(FILENAME + 'data.pkl', 'wb') )
    print('Success_rate : {}, average costs:{}'.format(data['success_rate'], np.mean(data['costs'])))
else:
    #load data
    data = pickle.load( open(FILENAME + 'data.pkl', 'rb'))
    x_init = data['x']
    y_init = data['y']
    num_traj = len(x_init)

### Reduce the data sample

In [33]:
#if necessary, reduce the size of database
x_init = x_init[0:200]
y_init = y_init[0:200]

### Build Function Approximator

In [34]:
dim_input = len(x_init[0])
is_load_regressor = False

straight = Straight_Regressor(dof = dof, n_steps=n_steps)
nn = NN_Regressor()
gpy = GPy_Regressor(dim_input = dim_input)
bgmr = DP_GLM_Regressor()

if is_load_regressor:
    nn.load_from_file(FILENAME + 'nn')
    gpy.load_from_file(FILENAME + 'gpy')
    bgmr.load_from_file(FILENAME + 'bgmr')
else:
    print 'Planning for NN'
    nn.fit(x_init, y_init)
    nn.save_to_file(FILENAME + 'nn')

    print 'Planning for GPY'
    gpy.fit(x_init, y_init, num_restarts=20)
    gpy.save_to_file(FILENAME + 'gpy')

    print 'Planning for BGMR'
    bgmr.fit(x_init,y_init)
    bgmr.save_to_file(FILENAME + 'bgmr')

Planning for NN
Planning for GPY
Optimization restart 1/20, f = 20843.237502
Optimization restart 2/20, f = 20843.2355451
Optimization restart 3/20, f = 20843.235711
Optimization restart 4/20, f = 20843.2391505
Optimization restart 5/20, f = 20843.2361392
Optimization restart 6/20, f = 20843.2358794
Optimization restart 7/20, f = 20843.2356344
Optimization restart 8/20, f = 20843.2381284
Optimization restart 9/20, f = 20843.2366081
Optimization restart 10/20, f = 20843.235585
Optimization restart 11/20, f = 20843.2355793
Optimization restart 12/20, f = 20843.2358112
Optimization restart 13/20, f = 20843.2435439
Optimization restart 14/20, f = 20843.2365999
Optimization restart 15/20, f = 20843.2361111
Optimization restart 16/20, f = 20843.2356962
Optimization restart 17/20, f = 20843.2365215
Optimization restart 18/20, f = 20843.2355835
Optimization restart 19/20, f = 20843.2356078
Optimization restart 20/20, f = 20843.236928
Planning for BGMR


### Creating the test cases

In [35]:
create_test = False
if create_test:
    x_test = []
    target_joints = []
    num_test = 1000
    for i in range(num_test):
        init_joint = generate_random_between(limits['init'][0], limits['init'][1])
        target_joint = generate_random_between(limits['goal'][0], limits['goal'][1])
        x_test.append(np.concatenate([init_joint, target_joint]))

    x_test = np.vstack(x_test)
    data_test = dict()
    data_test['x_test'] = x_test
    test_file = open(FILENAME + 'data_test.pkl', 'wb')
    pickle.dump(data_test,test_file)
    test_file.close()
else:
    #load data
    test_file = open(FILENAME + 'data_test.pkl', 'rb')
    data_test = pickle.load(test_file)
    x_test = data_test['x_test']
    num_test = len(x_test)
    test_file.close()

### Testing the warmstart

In [36]:
method_names = ['STD    ', 'NN     ', 'GPY    ',  'BGMR    ']
methods = [straight, nn, gpy,  bgmr]
results = dict()
result,success, duration = dict(),dict(),dict()
for method in method_names:
    results[method] = dict()
    results[method]['costs'] = []
    results[method]['successes'] = []
    results[method]['comp_times'] = []
    results[method]['func_evals'] = []
    results[method]['qp_solves'] = []
    
num_test = 100
for i in range(num_test):
    print('{}th Planning'.format(i))
    
    #setting up the problem case
    index = np.random.randint(0,len(x_test))
    index = i+100
    x_cur = x_test[index:index+1,:]
    init_joint = x_cur.copy().flatten()[0:3]
    target_joint = x_cur.copy().flatten()[3:]
    robot.SetActiveDOFValues(init_joint)
    
    #plan with various warmstarts
    request_traj = define_request(time_step=n_steps, coeffs = coeffs,init_type='given_traj',constraint_type='joint')
    request_traj = add_constraint(request_traj, 'joint', '', target_joint,-1)
    
    for i,method in enumerate(methods):
        method_name = method_names[i]
        if isinstance(method, Straight_Regressor):
            traj,cov = method.predict_with_waypoint(init_joint, target_joint, waypoint_right, n_steps/2)
        else:
            traj,cov = method.predict(x_cur)
            
        if method.is_transform == 'PCA':
            traj = y_pca.inverse_transform(traj)
            
        traj = traj.reshape(-1,dof)
        traj[0,:] = init_joint
        robot.SetActiveDOFValues(init_joint)
        request_traj = set_init(request_traj, 'given_traj', traj)
        duration[method_name], result[method_name] = run_opt(request_traj, env)    
        success[method_name] = check_traj(env,result[method_name], target_joint)

    #Record the result
    for method_name in method_names:
        results[method_name]['costs'].append(result[method_name].GetCosts()[0][1])
        results[method_name]['func_evals'].append(result[method_name].GetNumFuncEval()[0])
        results[method_name]['qp_solves'].append(result[method_name].GetNumQPSolve()[0])
        results[method_name]['successes'].append(success[method_name])      
        results[method_name]['comp_times'].append(duration[method_name])


0th Planning
Optimization is completed in 0.780693054199 s!
Optimization is completed in 0.137155056 s!
Optimization is completed in 1.52529788017 s!
There is a collision within the trajectory!
Optimization is completed in 0.303962945938 s!
1th Planning
Optimization is completed in 0.680097103119 s!
Optimization is completed in 0.197221040726 s!
Optimization is completed in 1.45822501183 s!
There is a collision within the trajectory!
Optimization is completed in 0.372343063354 s!
2th Planning
Optimization is completed in 0.688657045364 s!
Optimization is completed in 0.3262591362 s!
Optimization is completed in 1.53260803223 s!
There is a collision within the trajectory!
Optimization is completed in 0.218670129776 s!
3th Planning
Optimization is completed in 0.50377202034 s!
Optimization is completed in 0.29879117012 s!
Optimization is completed in 0.745807886124 s!
There is a collision within the trajectory!
Optimization is completed in 0.206791877747 s!
4th Planning
Optimization is c

### Printing Results

In [37]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
for method in method_names:
    successes = np.array(results[method]['successes'])
    success = np.count_nonzero(successes)
    
    comp_times = np.array(results[method]['comp_times'])[successes]
    costs = np.array(results[method]['costs'])[successes]
    func_evals = np.array(results[method]['func_evals'])[successes]
    qp_solves = np.array(results[method]['qp_solves'])[successes]
    
    print('{0} \t& {1:.3f} \t& {2:.2f} \t& {3:.3f} \t \\\\'.format(method, success*100.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success))#, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    

 Method 	| Success Rate 	| Conv. Time 	| Traj. Cost 	| Func. Evals 	| QP Solves
STD     	& 79.000 	& 0.53 	& 1.368 	 \\
NN      	& 97.000 	& 0.35 	& 1.417 	 \\
GPY     	& 0.000 	& nan 	& nan 	 \\
BGMR     	& 94.000 	& 0.32 	& 1.331 	 \\


 /home/teguh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning:invalid value encountered in double_scalars


### Save Result

In [31]:
pickle.dump(results,open(FILENAME + 'result_both.pkl', 'wb') )

### Load Result 

In [ ]:
results = pickle.load(open(FILENAME + 'result_both.pkl', 'rb'))

### Printing Results Tied to Standard Success

In [ ]:
print(' Method \t| Success Rate \t| Conv. Time \t| Traj. Cost \t| Func. Evals \t| QP Solves')
standard_successes = np.array(results['STD    ']['successes'])

for method in method_names:
    successes = np.array(results[method]['successes'])[standard_successes]
    success = np.count_nonzero(successes)
    comp_times = np.array(results[method]['comp_times'])[standard_successes][successes]
    costs = np.array(results[method]['costs'])[standard_successes][successes]
    func_evals = np.array(results[method]['func_evals'])[standard_successes][successes]
    qp_solves = np.array(results[method]['qp_solves'])[standard_successes][successes]
    
    print('{0}: \t {1:.3f} \t {2:.2f} \t {3:.3f} \t {4:.3f} \t {5:.3f}'.format(method, success*1.0/len(successes), np.sum(comp_times)/success, np.sum(costs)/success, 1.0*np.sum(func_evals)/success, 1.0*np.sum(qp_solves)/success ))
    